In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from collections import Counter

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

n_children = 1000000 # n children to give
n_gift_type = 1000 # n types of gifts available
n_gift_quantity = 1000 # each type of gifts are limited to this quantity
n_gift_pref = 100 # number of gifts a child ranks
n_child_pref = 1000 # number of children a gift ranks
twins = math.ceil(0.04 * n_children / 2.) * 2    # 4% of all population, rounded to the closest number
triplets = math.ceil(0.005 * n_children / 3.) * 3    # 0.5% of all population, rounded to the closest number
ratio_gift_happiness = 2
ratio_child_happiness = 2



gift_pref = pd.read_csv('../input/child_wishlist_v2.csv',header=None).drop(0, 1).values
child_pref = pd.read_csv('../input/gift_goodkids_v2.csv',header=None).drop(0, 1).values


def lcm(a, b):
    """Compute the lowest common multiple of a and b"""
    # in case of large numbers, using floor division
    return a * b // math.gcd(a, b)

def avg_normalized_happiness(pred, child_pref, gift_pref):
    
    # check if number of each gift exceeds n_gift_quantity
    gift_counts = Counter(elem[1] for elem in pred)
    for count in gift_counts.values():
        assert count <= n_gift_quantity
                
    # check if triplets have the same gift
    for t1 in np.arange(0,triplets,3):
        triplet1 = pred[t1]
        triplet2 = pred[t1+1]
        triplet3 = pred[t1+2]
        # print(t1, triplet1, triplet2, triplet3)
        assert triplet1[1] == triplet1[1] and triplet2[1] == triplet3[1]
                
    # check if twins have the same gift
    for t1 in np.arange(triplets,triplets+twins,2):
        twin1 = pred[t1]
        twin2 = pred[t1+1]
        # print(t1)
        assert twin1[1] == twin2[1]

    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for row in pred:
        child_id = row[0]
        gift_id = row[1]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(child_pref[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
    
    print('normalized child happiness=',float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) , \
        ', normalized gift happiness',np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity))

    # to avoid float rounding error
    # find common denominator
    # NOTE: I used this code to experiment different parameters, so it was necessary to get the multiplier
    # Note: You should hard-code the multipler to speed up, now that the parameters are finalized
    denominator1 = n_children*max_child_happiness
    denominator2 = n_gift_quantity*max_gift_happiness*n_gift_type
    common_denom = lcm(denominator1, denominator2)
    multiplier = common_denom / denominator1

    # # usually denom1 > demon2
    return float(math.pow(total_child_happiness*multiplier,3) + math.pow(np.sum(total_gift_happiness),3)) / float(math.pow(common_denom,3))
    # return math.pow(float(total_child_happiness)/(float(n_children)*float(max_child_happiness)),2) + math.pow(np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity),2)

random_sub = pd.read_csv('../input/sample_submission_random_v2.csv').values.tolist()
print(avg_normalized_happiness(random_sub, child_pref, gift_pref))

child_wishlist.csv
child_wishlist_v2.csv
gift_goodkids.csv
gift_goodkids_v2.csv
sample_submission_random.csv
sample_submission_random_v2.csv
santa-gift-matching-publicleaderboard.csv

normalized child happiness= 0.04621203 , normalized gift happiness -4.59355e-05
9.868817990273061e-05


In [2]:
gift_pref.shape, child_pref.shape

((1000000, 100), (1000, 1000))

In [33]:
class Child(object):
    
    def __init__(self, idx, prefer):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for i in range(prefer.shape[0]):
            self.prefer_dict[prefer[i]] = [12*(prefer.shape[0] - i), -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 6*score
        else:
            self.prefer_dict[giftid] = [-6, 6*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])
    
class Child_twin(object):
    
    def __init__(self, idx, prefer1, prefer2):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2))):
            score = 0
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = [3*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 3*score
        else:
            self.prefer_dict[giftid] = [-6, 3*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])
    
class Child_triplet(object):
    
    def __init__(self, idx, prefer1, prefer2, prefer3):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2) + list(prefer3))):
            score = 0
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            if p in list(prefer3):
                score += 2*(100 - list(prefer3).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = [2*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 2*score
        else:
            self.prefer_dict[giftid] = [-6, 2*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])
    
    
Children = []
for i in range(0, 5001, 3):
    Children.append(Child_triplet(i, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+1, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+2, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
for i in range(5001, 45001, 2):
    Children.append(Child_twin(i, gift_pref[i], gift_pref[i+1]))
    Children.append(Child_twin(i+1, gift_pref[i], gift_pref[i+1]))
Children = Children + [Child(i, gift_pref[i]) for i in range(45001, 1000000)]

for j in range(1000):
    cf = child_pref[j]
    done_list = []
    for i in range(cf.shape[0]):
        if cf[i] <= 5000 and cf[i] not in done_list:
            if cf[i] % 3 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                cid3 = cf[i] + 2
                done_list.append(cid2)
                done_list.append(cid3)
            elif cf[i] % 3 == 1:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                cid3 = cf[i] + 1
                done_list.append(cid1)
                done_list.append(cid3)
            else:
                cid1 = cf[i] - 2
                cid2 = cf[i] - 1
                cid3 = cf[i]
                done_list.append(cid1)
                done_list.append(cid2)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            if cid3 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid3))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
            Children[cid3].add_gifts_prefer(j, score_)
        elif cf[i] <= 45000 and cf[i] not in done_list:
            if cf[i] % 2 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                done_list.append(cid2)
            else:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                done_list.append(cid1)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
        elif cf[i] > 45000:
            Children[cf[i]].add_gifts_prefer(j, 2*(cf.shape[0] - i))

In [57]:
from ortools.graph import pywrapgraph

W_CHILD = 49184482
W_GIFTS = 1

In [58]:
11739596260**2 / 1678116**2    

48939819.81400522

In [59]:
min_cost_flow_1 = pywrapgraph.SimpleMinCostFlow()

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []


# triplets
for i in range(0, 5001, 3):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(3)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# triplets
for i in range(5001, 45001, 2):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(2)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# other children
for i in range(45001, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))

# add Arc
# gift -> children
for i in range(len(start_nodes)):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
    
# children -> 1001000 : collection
for i in range(0, 5001, 3):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(3), int(0)
    )
for i in range(5001, 45001, 2):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(2), int(0)
    )
for i in range(45001, 1000000):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(1), int(0)
    )
    
# gift -> 1001001 : dust_gift
for i in range(1000):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1001001), int(1000), int(0)
    )
    
# 1001001 -> 1001000 : dust_path
min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(1001001), int(1001000), int(1000000), int(0)
    )


# add Supply
for i in range(1000):
    min_cost_flow_1.SetNodeSupply(int(1000000+i), int(1000))

# children
for i in range(0, 5001, 3):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(5001, 45001, 2):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(45001, 1000000):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))

min_cost_flow_1.SetNodeSupply(int(1001001), int(0)) 
min_cost_flow_1.SetNodeSupply(int(1001000), int(-1000000)) 



In [60]:
min_cost_flow_1.Solve()

1

In [61]:
assignment = [-1]*1000000
twins_differ = []
triplets_differ = []

for i in range(min_cost_flow_1.NumArcs()):
    if min_cost_flow_1.Flow(i) != 0 and min_cost_flow_1.Head(i) < 1000000:
        c = min_cost_flow_1.Head(i)
        g = min_cost_flow_1.Tail(i)
        f = min_cost_flow_1.Flow(i)

        if c >= 45001:
            assignment[c] = g - 1000000

        elif c >= 5001:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    twins_differ.append([c, c+1])
                else:
                    assignment[c+1] = g - 1000000
            elif f == 2:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
        else:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                elif assignment[c+1] == -1:
                    assignment[c+1] = g - 1000000
                else:
                    assignment[c+2] = g - 1000000
            elif f == 2:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    assignment[c+1] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                else:
                    assignment[c+1] = g - 1000000
                    assignment[c+2] = g - 1000000
            elif f == 3:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
                assignment[c+2] = g - 1000000
                
CHILD_HAPPINESS = sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(W_CHILD, W_GIFTS, CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))

49184482 1 11739596260 1678116 1617929090200058768912337440896
0.9363015568287377


In [ ]:
# 1 0 11739596260 416365 1617929090195405252193199803125
# 0.9363015568260447
# 794983092 1 11739596260 1673937 1617929090200023551648334443953
# 0.9363015568287173
# 49184482 1 11739596260 1678116 1617929090200058768912337440896
# 0.9363015568287377
# 48939820 1 11739596260 1673937 1617929090200023551648334443953
# 0.9363015568287173
# 49184482 1 11739596260 1678116 1617929090200058768912337440896
# 0.9363015568287377

wata: 0.9362995374

In [81]:
def loss_change(i, j):
    return 1*(Children[i].happiness(assignment[i])[0] - Children[i].happiness(j)[0]) + 0*(Children[i].happiness(assignment[i])[1] - Children[i].happiness(j)[1])

In [62]:
len(twins_differ), len(triplets_differ)

(94, 11)

In [63]:
Gifts_left = [1000 for _ in range(1000)]
for i in range(1000000):
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1

In [64]:
for i in range(1000):
    if Gifts_left[i] != 0:
        print(i, Gifts_left[i])

118 628
240 178
272 143
320 496
389 150
494 981
671 707
998 409


In [65]:
[118, 240, 272, 320, 389, 494, 671, 998]

[118, 240, 272, 320, 389, 494, 671, 998]

In [66]:
unassigned_v = [0, 0, 0]
unassigned_v_ = [0, 0]
for i in range(5001):
    if assignment[i] == -1:
        unassigned_v[0] += 1
for i in range(5001, 45001):
    if assignment[i] == -1:
        unassigned_v[1] += 1
for i in range(45001, 1000000):
    if assignment[i] == -1:
        unassigned_v[2] += 1 
for i in range(0, 5001, 3):
    if assignment[i] == -1:
        unassigned_v_[0] += 1
for i in range(5001, 45001, 2):
    if assignment[i] == -1:
        unassigned_v_[1] += 1

In [67]:
unassigned_v, unassigned_v_

([2751, 941, 0], [912, 455])

In [68]:
912*3, 455*2

(2736, 910)

In [71]:
[118, 240, 272, 320, 389, 494, 671, 998]

[118, 240, 272, 320, 389, 494, 671, 998]

In [69]:
for tri in triplets_differ:
    print(tri, [assignment[tri[0]], assignment[tri[1]], assignment[tri[2]]])

[1413, 1414, 1415] [38, 402, 402]
[1467, 1468, 1469] [466, -1, -1]
[1647, 1648, 1649] [463, 463, 504]
[1692, 1693, 1694] [462, -1, -1]
[1791, 1792, 1793] [402, -1, -1]
[1950, 1951, 1952] [218, -1, -1]
[3675, 3676, 3677] [637, -1, -1]
[3897, 3898, 3899] [204, 204, -1]
[3987, 3988, 3989] [248, 248, -1]
[4590, 4591, 4592] [759, 759, -1]
[4743, 4744, 4745] [907, -1, -1]


In [70]:
assignment_save = assignment.copy()

In [91]:
# assignment = assignment_save.copy()

In [93]:
triplets_differ_1 = []
triplets_differ_2 = []
triplets_differ_3 = []

for tri in triplets_differ:
    if assignment[tri[2]] == -1:
        if assignment[tri[1]] == -1:
            triplets_differ_2.append(tri)
        else:
            triplets_differ_1.append(tri)
    else:
        if assignment[tri[0]] == assignment[tri[1]]:
            triplets_differ_3.append(tri)
        else:
            assignment[tri[0]], assignment[tri[2]] = assignment[tri[2]], assignment[tri[0]]
            triplets_differ_3.append(tri)
            

In [95]:
len(triplets_differ_1), len(triplets_differ_2), len(triplets_differ_3)

(3, 6, 2)

In [96]:
# [X, -1, -1]
loss_sum = 0
for tri in triplets_differ_2:
    g_tri = assignment[tri[0]]
    min_loss_1_1 = 1000000
    # Gomi -> X
    for i in range(45001, 1000000):
        if assignment[i] in [118, 240, 272, 320, 389, 494, 671, 998]:
            loss_ = loss_change(i, g_tri)
            if loss_ < min_loss_1_1:
                min_loss_1_1 = loss_
                i_m_1 = i
    min_loss_1_2 = 1000000
    j_m_1 = 118
    for j in [118, 240, 272, 320, 389, 494, 671, 998]:
        loss_ = loss_change(tri[0], j)
        if loss_ < min_loss_1_2:
            min_loss_1_2 = loss_
            j_m_1 = j
    min_loss_1 = min_loss_1_1 + min_loss_1_2
    
    # [X, X, X]
    i_list = []
    loss_2_1_list = []
    j_m_2_list = []
    for i in range(45001, 1000000):
        if assignment[i] == g_tri:
            i_list.append(i)
            loss_2_1 = 1000000
            j_m_2 = 118
            for j in [118, 240, 272, 320, 389, 494, 671, 998]:
                loss_ = loss_change(i, j)
                if loss_ < loss_2_1:
                    loss_2_1 = loss_
                    j_m_2 = j
            loss_2_1_list.append(loss_2_1)
            j_m_2_list.append(j_m_2)
    min_los2_arg = np.argsort(loss_2_1_list)
    loss_2 = loss_2_1_list[min_los2_arg[0]] + loss_2_1_list[min_los2_arg[1]] + 2*loss_change(tri[2], g_tri)
    
    # switch
    if loss_2 >= min_loss_1:
        loss_sum += min_loss_1
        assignment[i_m_1] = g_tri
        assignment[tri[0]] = j_m_1
        assignment[tri[1]] = j_m_1
        assignment[tri[2]] = j_m_1
    else:
        loss_sum += loss_2
        assignment[i_list[min_los2_arg[0]]] = j_m_2_list[min_los2_arg[0]]
        assignment[i_list[min_los2_arg[1]]] = j_m_2_list[min_los2_arg[1]]
        assignment[tri[1]] = g_tri
        assignment[tri[2]] = g_tri
    print(min(min_loss_1, loss_2))

54
78
42
78
78
66


In [97]:
# [X, X, -1]
for tri in triplets_differ_1:
    g_tri = assignment[tri[0]]
    # Gomi -> X
    min_loss_1_1_list = []
    i_list = []
    for i in range(45001, 1000000):
        if assignment[i] in [118, 240, 272, 320, 389, 494, 671, 998]:
            loss_ = loss_change(i, g_tri)
            i_list.append(i)
            min_loss_1_1_list.append(loss_)
    min_loss_1_2 = 1000000
    j_m_1 = 118
    for j in [118, 240, 272, 320, 389, 494, 671, 998]:
        loss_ = loss_change(tri[0], j)
        if loss_ < min_loss_1_2:
            min_loss_1_2 = loss_
            j_m_1 = j
    min_los1_arg = np.argsort(min_loss_1_1_list)
    min_loss_1 = min_loss_1_1_list[min_los1_arg[0]] + min_loss_1_1_list[min_los1_arg[1]] + 2*min_loss_1_2
    
    # [X, X, X]
    i_m_2 = 0
    loss_2_1 = 1000000
    j_m_2 = 0
    for i in range(45001, 1000000):
        if assignment[i] == g_tri:
            for j in [118, 240, 272, 320, 389, 494, 671, 998]:
                loss_ = loss_change(i, j)
                if loss_ < loss_2_1:
                    loss_2_1 = loss_
                    j_m_2 = j
                    i_m_2 = i
                    
    loss_2 = loss_2_1 + loss_change(tri[2], g_tri)
    
    # switch
    if loss_2 >= min_loss_1:
        loss_sum += min_loss_1
        assignment[i_list[min_los1_arg[0]]] = g_tri
        assignment[i_list[min_los1_arg[1]]] = g_tri
        assignment[tri[0]] = j_m_1
        assignment[tri[1]] = j_m_1
        assignment[tri[2]] = j_m_1
    else:
        loss_sum += loss_2
        assignment[i_m_2] = j_m_2
        assignment[tri[2]] = g_tri
    print(min(min_loss_1, loss_2))

160
132
66


In [100]:
# [X, X, Y]
for tri in triplets_differ_3:
    min_loss = 1000000
    
    t0 = tri[0]
    t1 = tri[2]
    flg = 0
    
    gid = assignment[t0]
    g = assignment[t1]
    for i in range(45001, 1000000):
        if assignment[i] == gid:
            loss_ = loss_change(i, g) + loss_change(t1, gid)
            if loss_ < min_loss:
                min_loss = loss_
                i_m = i
                    
    gid = assignment[t1]
    g = assignment[t0]
    loss_list = []
    i_list = []
    for i in range(45001, 1000000):
        if assignment[i] == gid:
            i_list.append(i)
            loss_ = loss_change(i, g) + loss_change(t1, gid)
            loss_list.append(loss_)
            
    loss_list_s = np.argsort(loss_list)
    
    if loss_list[loss_list_s[0]] + loss_list[loss_list_s[1]] < min_loss:
        min_loss = loss_list[loss_list_s[0]] + loss_list[loss_list_s[1]]
        i_m_1 = i_list[loss_list_s[0]]
        i_m_2 = i_list[loss_list_s[1]]
        flg = 1
        
                    
    print(min_loss)
    loss_sum += min_loss
    
    if flg == 0:
        assignment[t1], assignment[i_m] = assignment[i_m], assignment[t1]
    else:
        assignment[t0], assignment[t0+1], assignment[i_m_1], assignment[i_m_2] = assignment[i_m_1], assignment[i_m_2], assignment[t0], assignment[t0+1]

12
12


In [101]:
print(loss_sum)

778


In [102]:
assignment_save_2 = assignment.copy()

In [103]:
twins_differ_1 = []
twins_differ_2 = []

for twi in twins_differ:
    if assignment[twi[1]] == -1:
        twins_differ_1.append(twi)
    else:
        twins_differ_2.append(twi)

In [104]:
Gifts_left = [1000 for _ in range(1000)]
for i in range(1000000):
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1
for i in range(1000):
    if Gifts_left[i] != 0:
        print(i, Gifts_left[i])

118 607
240 179
272 144
320 497
389 151
494 981
671 707
998 411


In [105]:
len(twins_differ_1), len(twins_differ_2)

(31, 63)

In [106]:
# [X, -1]
for twi in twins_differ_1:
    g_twi = assignment[twi[0]]
    min_loss_1_1 = 1000000
    # Gomi -> X
    for i in range(45001, 1000000):
        if assignment[i] in [118, 240, 272, 320, 389, 494, 671, 998]:
            loss_ = Children[i].happiness(assignment[i])[0] - Children[i].happiness(g_twi)[0]
            if loss_ < min_loss_1_1:
                min_loss_1_1 = loss_
                i_m_1 = i
    min_loss_1_2 = 1000000
    j_m_1 = 118
    for j in [118, 240, 272, 320, 389, 494, 671, 998]:
        loss_ = Children[twi[0]].happiness(g_twi)[0] - Children[twi[0]].happiness(j)[0]
        if loss_ < min_loss_1_2:
            min_loss_1_2 = loss_
            j_m_1 = j
    min_loss_1 = min_loss_1_1 + min_loss_1_2
    
    # [X, X]
    i_m_2 = 0
    loss_2_1 = 1000000
    j_m_2 = 0
    for i in range(45001, 1000000):
        if assignment[i] == g_twi:
            for j in [118, 240, 272, 320, 389, 494, 671, 998]:
                loss_ = Children[i].happiness(g_twi)[0] - Children[i].happiness(j)[0]
                if loss_ < loss_2_1:
                    loss_2_1 = loss_
                    j_m_2 = j
                    i_m_2 = i
                    
    loss_2 = loss_2_1 + (Children[twi[0]].happiness(-1)[0] - Children[twi[0]].happiness(g_twi)[0])
    
    # switch
    if loss_2 >= min_loss_1:
        loss_sum += min_loss_1
        assignment[i_m_1] = g_twi
        assignment[twi[0]] = j_m_1
        assignment[twi[1]] = j_m_1
    else:
        loss_sum += loss_2
        assignment[i_m_2] = j_m_2
        assignment[twi[1]] = g_twi
    print(min(min_loss_1, loss_2))

6
30
42
102
6
30
18
18
42
6
42
114
18
54
54
42
30
6
54
42
42
18
66
42
6
66
42
30
18
54
30


In [107]:
Gifts_left = [1000 for _ in range(1000)]
for i in range(1000000):
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1
for i in range(1000):
    if Gifts_left[i] != 0:
        print(i, Gifts_left[i])

118 566
240 184
272 148
320 497
389 151
494 981
671 706
998 413


In [108]:
assignment_save_3 = assignment.copy()

In [110]:
# assignment = assignment_save_3.copy()

In [112]:
for twi in twins_differ_2:
    min_loss = 1000000
    
    t0 = twi[0]
    t1 = twi[1]
    flg = 0
    
    gid = assignment[t0]
    g = assignment[t1]
    for i in range(45001, 1000000):
        if assignment[i] == gid:
            loss_ = Children[i].happiness(assignment[i])[0] - Children[i].happiness(g)[0] + Children[t1].happiness(g)[0] - Children[t1].happiness(gid)[0]
            if loss_ < min_loss:
                min_loss = loss_
                i_m = i
                    
    gid = assignment[t1]
    g = assignment[t0]
    for i in range(45001, 1000000):
        if assignment[i] == gid:
            loss_ = Children[i].happiness(assignment[i])[0] - Children[i].happiness(g)[0] + Children[t0].happiness(g)[0] - Children[t0].happiness(gid)[0]
            if loss_ < min_loss:
                min_loss = loss_
                i_m = i
                flg = 1
    print(min_loss)
    loss_sum += min_loss
    if flg == 0:
        assignment[t1], assignment[i_m] = assignment[i_m], assignment[t1]
    else:
        assignment[t0], assignment[i_m] = assignment[i_m], assignment[t0]

12
12
12
24
12
0
12
0
12
12
12
0
0
12
12
12
0
18
24
12
12
0
12
24
0
0
0
12
24
6
12
12
12
72
12
12
0
12
0
12
12
0
0
0
12
12
0
12
6
6
12
12
12
0
0
0
12
36
12
0
0
12
0


In [113]:
# check twins and triplets
for i in range(0, 5001, 3):
    if assignment[i] == assignment[i+1] and assignment[i] == assignment[i+2]:
        pass
    else:
        print(i)
for i in range(5001, 45001, 2):
    if assignment[i] != assignment[i+1]:
        print(i)

In [132]:
loss_sum

2572

In [133]:
((11739596260-25720) / 12000000000)**3

0.9362954028775838

In [120]:
unassigned_v = [0, 0, 0]
unassigned_v_ = [0, 0]
for i in range(5001):
    if assignment[i] == -1:
        unassigned_v[0] += 1
for i in range(5001, 45001):
    if assignment[i] == -1:
        unassigned_v[1] += 1
for i in range(45001, 1000000):
    if assignment[i] == -1:
        unassigned_v[2] += 1 
for i in range(0, 5001, 3):
    if assignment[i] == -1:
        unassigned_v_[0] += 1
for i in range(5001, 45001, 2):
    if assignment[i] == -1:
        unassigned_v_[1] += 1

In [121]:
unassigned_v, unassigned_v_

([2736, 910, 0], [912, 455])

In [119]:
Gifts_left = [1000 for _ in range(1000)]
for i in range(1000000):
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1
for i in range(1000):
    if Gifts_left[i] != 0:
        print(i, Gifts_left[i])

118 566
240 184
272 148
320 497
389 151
494 981
671 706
998 413


In [125]:
gifts_for_trips = [566, 184, 148, 497, 151, 981, 706, 413]
gifts_for_twins = [2, 4, 4, 2, 4, 888, 4, 2]
for i in range(8):
    gifts_for_trips[i] -= gifts_for_twins[i]

In [128]:
gifts_left_list = []
for i in range(gifts_for_trips[0]):
    gifts_left_list.append(118)
for i in range(gifts_for_trips[1]):
    gifts_left_list.append(240)
for i in range(gifts_for_trips[2]):
    gifts_left_list.append(272)
for i in range(gifts_for_trips[3]):
    gifts_left_list.append(320)
for i in range(gifts_for_trips[4]):
    gifts_left_list.append(389)
for i in range(gifts_for_trips[5]):
    gifts_left_list.append(494)
for i in range(gifts_for_trips[6]):
    gifts_left_list.append(671)
for i in range(gifts_for_trips[7]):
    gifts_left_list.append(998)
    
for i in range(gifts_for_twins[0]):
    gifts_left_list.append(118)
for i in range(gifts_for_twins[1]):
    gifts_left_list.append(240)
for i in range(gifts_for_twins[2]):
    gifts_left_list.append(272)
for i in range(gifts_for_twins[3]):
    gifts_left_list.append(320)
for i in range(gifts_for_twins[4]):
    gifts_left_list.append(389)
for i in range(gifts_for_twins[5]):
    gifts_left_list.append(494)
for i in range(gifts_for_twins[6]):
    gifts_left_list.append(671)
for i in range(gifts_for_twins[7]):
    gifts_left_list.append(998)

In [129]:
assignment_save_4 = assignment.copy()

In [130]:
ind = 0
for i in range(1000000):
    if assignment[i] == -1:
        assignment[i] = gifts_left_list[ind]
        ind += 1

In [131]:
out = open('../submit/subm_hrd_new_3.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(1000000):
    out.write(str(i) + ',' + str(assignment[i]) + '\n')
out.close()

In [83]:
len(gifts_left_list)

3761

In [73]:
sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10

11739493500

In [74]:
sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])

125802

In [75]:
((11739493500**3) + (125802**3)) / (12000000000**3)

0.9362769699057234

In [65]:
unassigned_v = [0, 0, 0]
unassigned_v_ = [0, 0]
for i in range(5001):
    if assignment[i] == -1:
        unassigned_v[0] += 1
for i in range(5001, 45001):
    if assignment[i] == -1:
        unassigned_v[1] += 1
for i in range(45001, 1000000):
    if assignment[i] == -1:
        unassigned_v[2] += 1 
for i in range(0, 5001, 3):
    if assignment[i] == -1:
        unassigned_v_[0] += 1
for i in range(5001, 45001, 2):
    if assignment[i] == -1:
        unassigned_v_[1] += 1

In [66]:
unassigned_v, unassigned_v_

([2835, 926, 0], [945, 463])

In [67]:
sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10

11739499020

In [68]:
sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])

125802

In [69]:
((11739499020**3) + (125802**3)) / (12000000000**3)

0.9362782906402093